In [1]:
#Importing relevant packages for training

import math
import pandas as pd
import numpy as np

In [2]:
#Importing beer.txt from local machine and adding labels and also for flexibility if there is another dataset with similar
# type attributes these lists could be changed.

full_dataset = pd.read_csv('C:/Users/josho/OneDrive/Documents/Msc_AI_Sem_1/ML/Assignment_2/Beer.txt', delimiter="\t", header = None)
full_dataset.columns = ['calorific_value', 'nitrogen', 'turbidity', 'style', 'alcohol', 'sugars', 'bitterness', 'beer_id', 'colour', 'degree_of_fermentation']
full_dataset

,calorific_value,nitrogen,turbidity,style,alcohol,sugars,bitterness,beer_id,colour,degree_of_fermentation
0,41.721239,0.503276,2.628182,ale,4.015385,16.73,10.452789,93,13.44,55.337143
1,42.429204,0.525512,1.776364,ale,4.092308,16.72,10.999526,103,12.24,58.380000
2,45.880531,0.443233,2.628182,ale,4.276923,16.68,13.456368,178,10.92,58.382857
3,45.305310,0.471668,1.806364,ale,4.126154,18.84,9.202737,166,10.92,58.525714
4,38.977876,0.392846,2.272727,ale,4.015385,16.77,9.457895,44,10.56,58.900000
...,...,...,...,...,...,...,...,...,...,...
149,39.951327,0.272587,5.217273,stout,4.412308,16.68,6.008368,54,6.84,74.757143
150,40.039823,0.347919,2.859091,stout,4.178462,17.58,7.340842,56,8.88,74.887143
151,43.977876,0.266770,3.775455,stout,4.347692,19.10,7.356000,132,7.32,75.894286
152,45.349558,0.255530,1.301818,stout,4.160000,18.17,3.243579,169,10.68,76.182857


In [3]:
#Adding a lists of the dependent and independent var(s) for future convinience

X_vars = ['calorific_value', 'nitrogen', 'turbidity', 'alcohol', 'sugars', 'bitterness', 'beer_id', 'colour', 'degree_of_fermentation']
y_var = 'style'

In [4]:
#Working with numerical attributes and so thresholds need to be defined,function took the mean as these thresholds

def get_thresholds(df, x_labs):
    
    thresholds = {att : np.mean(df[att]) for att in x_labs}  #Dictonary comprehension to store each mean
    return thresholds

#test

get_thresholds(full_dataset, X_vars)


{'calorific_value': 41.654579933506476,
 'nitrogen': 0.3208729250584415,
 'turbidity': 1.8850767390974037,
 'alcohol': 4.0061138861233765,
 'sugars': 17.335909090909087,
 'bitterness': 8.843541011305195,
 'beer_id': 90.36363636363636,
 'colour': 11.338129870129873,
 'degree_of_fermentation': 65.96386827435063}

In [5]:
# Funtion to create a boolean dataframe based on the inputted datafram and threshold dictionary

def get_boolean(df, y_lab, threshold_dict):
    dict_bool = {}
    for att in threshold_dict:
        dict_bool[att] = df[att] > threshold_dict[att] 
        
    dict_bool[y_lab] = df[y_lab]  #Need to add back in the dependant variable in order to calculate entropies

    return pd.DataFrame(dict_bool) #changing this dictionary back to a dataframe

#test
get_boolean(full_dataset, y_var, get_thresholds(full_dataset, X_vars))

,calorific_value,nitrogen,turbidity,alcohol,sugars,bitterness,beer_id,colour,degree_of_fermentation,style
0,True,True,True,True,False,True,True,True,False,ale
1,True,True,False,True,False,True,True,True,False,ale
2,True,True,True,True,False,True,True,False,False,ale
3,True,True,False,True,True,True,True,False,False,ale
4,False,True,True,True,False,True,False,False,False,ale
...,...,...,...,...,...,...,...,...,...,...
149,False,False,True,True,False,False,False,False,True,stout
150,False,True,True,True,True,False,False,False,True,stout
151,True,False,True,True,True,False,True,False,True,stout
152,True,False,False,True,True,False,True,False,True,stout


In [6]:
# Defined a function to calculate entropy using lecture definition
def entropy(S):
    
    return sum([((-i/sum(S))*math.log(i/sum(S),2)) for i in S])

# Defined a function to calculate information gain using lecture definition
def info_gain(df, att):
    y = 'style'
    a = sum(df[df[att] == True][y].value_counts())/sum(df[y].value_counts())*entropy(df[df[att] == True][y].value_counts())
    b = sum(df[df[att] == False][y].value_counts())/sum(df[y].value_counts())*entropy(df[df[att] == False][y].value_counts())
    return entropy(df[y].value_counts()) - a - b

In [7]:
# Function to find the best informtion gain , df is this case will be the boolean dataframe as seen in the test below.
def best_info_gains(df, x_labs):
    info_gains = []
    for att in x_labs:
        info_gains.append(info_gain(df, att))
    best_gain = max(info_gains)
    best_att = x_labs[info_gains.index(best_gain)] # Locating the att using the index of the max information gain.

    return best_att, best_gain

#test
best_info_gains(get_boolean(full_dataset, 'style', get_thresholds(full_dataset, X_vars)), X_vars)

('nitrogen', 0.5367204219111416)

In [8]:
# Function which consists of some of the previous functions which finds the attribute with best information gain
# while only having access to the origina dataframe and the x, y label(s)
def find_best_attribute(df, x_labs, y_lab):
    th = get_thresholds(df, x_labs)
    boolean = get_boolean(df, y_lab, th)
    best_att, best_gain = best_info_gains(boolean, x_labs)
    
    return best_att, th[best_att]

In [9]:
# Function that splits the dataframe on the already found "best attribute" and its threshold into 2 dataframes 
def split(df, best_att, th):
    return df[df[best_att] < th], df[df[best_att] >= th]

In [21]:
#Function to create the tree
def tree(df, x_labs, y_lab, plan, leaves): # plan & leaves are both empty lists, plan eill hav each next decion made,
                                            # leaves will have the prediction value (style) at each leaf.

    best_att, best_th = find_best_attribute(df, x_labs, y_lab) # Record the best att and its threshold
    best = best_att, best_th
    plan.append(best)                                          # Record best which is essentially the next step
    left_node, right_node = split(df, best_att, best_th)       # split data using this att and threshold
    left_node = left_node.drop(best_att, axis = 1)         
    right_node = right_node.drop(best_att, axis = 1)           # Removing this already used att from the data
    xs = list(x_labs)
    xs.remove(best_att)                                        # Removing this already used att from the att list
    
    if len(left_node[y_lab].unique()) == 1:     # If there is only one unque value in style then we must be at a leaf
        #print(left_node[y_lab].unique()[0])
        plan.append('leaf')                                    # Letting the plan know we have reached a leaf
        leaves.append(left_node[y_lab].unique()[0]) # Letting the leaves know what style we are predicting at this specific leaf
            
    else:
        left_path = tree(left_node, xs, y_lab, plan, leaves) # If we havent reached a leaf repeat function on new data
            
    if len(right_node[y_lab].unique()) == 1:                     #
        #print(right_node[y_lab].unique()[0])                     #
        plan.append('leaf')                                      # 
        leaves.append(right_node[y_lab].unique()[0])             # SAME AS ABOVE
                                                                 #
    else:                                                        #
        right_path = tree(right_node, xs, y_lab, plan, leaves)   #
        
    return plan                                                  

In [11]:
# Function is to create the empty list plan and pass it into tree(), leaves we will see later
def train(df, x_labs, y_lab, leaves):
    plan = []
    return tree(df, x_labs, y_lab, plan, leaves)

In [12]:
#Needed to seperate data
from sklearn.model_selection import train_test_split

In [14]:
# Seperating data inline with the 2/3 to 1/3 requirement from the assignment brief
X_train, X_test, y_train, y_test = train_test_split(full_dataset[X_vars], full_dataset[y_var],test_size = 0.33, random_state = 1)

In [16]:
# Re-appending the dependant variable as this was how my functions were created.
X_train[y_var] = y_train
# Pre-processing I would carry out on an average dataset as beer_id would obvioulsy be a perfect dicriminator
X_train.drop('beer_id', axis = 1)

,calorific_value,nitrogen,turbidity,alcohol,sugars,bitterness,colour,degree_of_fermentation,style
45,44.508850,0.575723,2.030909,4.236923,16.90,12.543737,13.56,74.188571,ale
75,38.535398,0.286716,1.259091,3.793846,16.10,4.326368,15.00,62.560000,lager
39,41.455752,0.554016,2.251818,4.089231,16.97,10.486684,12.84,72.560000,ale
36,44.022124,0.495629,2.280000,4.255385,16.57,14.013684,13.56,71.054286,ale
119,42.473451,0.296829,2.902727,4.052308,20.19,3.847737,7.20,63.668571,stout
...,...,...,...,...,...,...,...,...,...
133,43.845133,0.311276,3.050000,4.218462,20.65,5.469947,7.68,67.150000,stout
137,40.880531,0.292846,3.717273,4.356923,17.51,4.064737,7.44,69.302857,stout
72,40.172566,0.141299,0.690909,3.393846,16.51,9.779632,20.52,61.914286,lager
140,41.500000,0.275861,2.229091,3.953846,18.27,4.228421,10.44,70.688571,stout


In [17]:
#Function to predict test data using already generated plan
def predict(df,y_lab, plan, true_vals):              # true vals will be a list of the actualy styles of each record
    next_decision = plan[0] 
    
    if next_decision == 'leaf': 
        true_vals.append(df[y_lab].value_counts())   # If at a leaf record true values at that leaf
        
    else:
        next_decision = plan[0]
        left_node, right_node  = split(df,next_decision[0] , next_decision[1]) # if not at leaf split data according to plan
        del plan[0]                                       # remove implemented decision
        predict(left_node,y_lab, plan, true_vals)         # predict again on child node
        del plan[0]                                       # same as above
        predict(right_node,y_lab, plan, true_vals)

In [18]:
# Again Re-appending y variable
X_test[y_var] = y_test

In [19]:
# Function to return how accurate the model is
def accuracy(training_df, testing_df, x_labs, y_lab):
    leaves = []
    true_vals = []
    correct = []
    total = []
    
    predict(testing_df,y_lab,train(training_df, x_labs, y_lab, leaves),true_vals)
    
    pairs = list(zip(leaves, true_vals))
    
    for pair in pairs:
        if pair[0] in pair[1]:
            correct.append(pair[1].loc[pair[0]])
        else:
            pass
        
    for pair in pairs:
        total.append(sum(pair[1]))
        
    return sum(correct)/sum(total)

In [22]:
accuracy(X_train,X_test, X_vars, y_var)

0.7647058823529411

In [25]:
accuracies = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(full_dataset[X_vars],
                                                        full_dataset[y_var],test_size = 0.33, random_state = i)
    X_train[y_var] = y_train
    X_test[y_var] = y_test
    X_train.drop('beer_id', axis = 1)
    accuracies.append(accuracy(X_train,X_test, X_vars, y_var))
    
accuracies

[0.7843137254901961,
 0.7647058823529411,
 0.8627450980392157,
 0.8431372549019608,
 0.8235294117647058,
 0.8823529411764706,
 0.8627450980392157,
 0.8627450980392157,
 0.7843137254901961,
 0.9019607843137255]

In [26]:
np.mean(accuracies)

0.8372549019607843